# Graphs of GRF, Inverse Kinematics, Dynamics and SO from XIMU3 using ROS-OpenSIMRT 



## ROT_STRING


In [ ]:
import os, sys
sys.path.append("/home/frekle/github/opensimrt/Data/refdata/refdata")
import refdata
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload

import glob
import logging
reload(logging)
from rich.logging import RichHandler
logging.basicConfig(format='%(funcName)s: %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S', 
                    handlers=[RichHandler()],
                    level=logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

import pprint
def myprint(var):
    nicevar = pprint.pformat(globals()[var], indent=1 , width=100)
    logger.debug(f"{var}:\n{nicevar}")


def myprint2(var):
    logger.debug(f"{var}:\n{globals()[var]}")
    
refdata.plt.rcParams['figure.figsize'] = [12, 5]
refdata.ROW_OF_FLOTS = 1

######################### TRIAL INFO ##########################################

this_action_name = "walking"
##Edit next line to select at least this action!
actions_to_be_shown = [ "walking" ]
#['walk','gait','slow','fast']

subject_num="02"
weight= 54
conv_names = refdata.graph_params.get_id_graph_params(weight)
myprint("conv_names")

skip_trials =[]

###############################################################################

#2023-03-03-11-56-24walking012_ik_lower.sto

#sto_files = glob.glob("/home/frekle/github/opensimrt/catkin_ws/tmp/02/id_so_batch_walking_calcn/*.sto")
#this is overkill. I know I want to make sure I am looking at the correct file, 
#but it was getting too difficult to debug. just be extra careful!

sto_files = glob.glob("*.sto")
sto_files.sort()
myprint("sto_files")

ik_files = []
grfL_files = []
grfR_files = []
id_files = []
for i in range(100): #this is the upper bound, but it doesnt matter unless there is a lot of deleted trials
    for file in sto_files:
        if "SCRIPT%d"%i in file:
            if "ik.sto" in file:
                ik_files.append(file)
                continue
            if "grfLeft.sto" in file:
                grfL_files.append(file)
                continue
            if "grfRight.sto" in file:
                grfR_files.append(file)
                continue
            if "tau.sto" in file:
                id_files.append(file)
                
                
myprint("ik_files")

myprint("grfL_files")

myprint("grfR_files")

myprint("id_files")

In [ ]:
reload(refdata)

In [ ]:
reload(refdata.graph_params)

## Finds the steps:

In [ ]:
step_seg_l_list = []
step_seg_r_list = []

for ik_, grfL_, grfR_ in zip(ik_files, grfL_files, grfR_files):   
    ik_2 = refdata.TrialData(ik_, remove_time_offset=False)
    zero_time = np.min(ik_2.data.time.values)
    step_seg_l_list.append(refdata.each_side_plot(grfL_,zero_time,grf_name_prefix = "1_ground_", side="Left", weight=weight))
    step_seg_r_list.append(refdata.each_side_plot(grfR_,zero_time,grf_name_prefix = "ground_", side="Right", weight=weight))

# GRF PLOTs

In [ ]:
action_trials_left= []
for trial in grfL_files:
    for action_name in actions_to_be_shown:
        if action_name in trial:
            action_trials_left.append(trial)
myprint("action_trials_left") #= ["/home/frekle/github/opensimrt/catkin_ws/tmp/02/48_s2_id_walking_filtered_1tau.sto"]
action_trials_right= []
for trial in grfR_files:
    for action_name in actions_to_be_shown:
        if action_name in trial:
            action_trials_right.append(trial)
myprint("action_trials_right") #= ["/home/frekle/github/opensimrt/catkin_ws/tmp/02/48_s2_id_walking_filtered_1tau.sto"]

###looks like I really didnt want to mess up left with right, so there are a ton of different checks. maybe a bit excessive
xy_idk_grfL = refdata.generate_somejoint_or_muscle_curves(action_trials_left,[],
                                                          curve_prefix="1_ground_force_vy",
                                                         conv_names= 
                                                          refdata.graph_params.generate_grf_conv_names(0,weight),
                                                          left_or_right=0)
xy_idk_grfR = refdata.generate_somejoint_or_muscle_curves(action_trials_right,[],
                                                          curve_prefix="ground_force_vy",
                                                         conv_names= 
                                                          refdata.graph_params.generate_grf_conv_names(1,weight),
                                                          left_or_right=1)

left_str_ = "xy_time_clippings_left = {\n"
for sg_list, (name, xy_tuples) in zip(step_seg_l_list,xy_idk_grfL[0].items()):
    left_str_ +="\"%s\":%s,\n"%(name,sg_list)
left_str_ += "}\n"
right_str_ = "xy_time_clippings_right = {\n"
for sg_list, (name, xy_tuples) in zip(step_seg_r_list,xy_idk_grfR[1].items()):
    right_str_ += "\"%s\":%s,\n"%(name,sg_list)
right_str_ += "}\n"

In [ ]:
refdata.create_new_cell_below(left_str_)

In [ ]:
refdata.create_new_cell_below(right_str_)

In [ ]:
##show some left side curve
if False: #True:
    for name, xy_tuples in xy_idk_grfL[0].items():
        if name == action_trials_left[2]:
            #plt.title(name)
            print(name)
            refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_left[name] )

##show some right side curve            
if False: #True:
    for name, xy_tuples in xy_idk_grfR[1].items():
        if name == action_trials_right[0]:
            #plt.title(name)
            print(name)
            refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_right[name] )
            
##This has to run

xy_clippings_both = (xy_time_clippings_left,xy_time_clippings_right)
print(xy_clippings_both[0])
print(xy_clippings_both[1])

## Shows all trials
if False: #True:
    for name, xy_tuples in xy_idk_grfL[0].items():    
        for action_name in actions_to_be_shown:
            if action_name in name:
                print(name+r" left")        
                refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_left[name] )
    for name, xy_tuples in xy_idk_grfR[1].items():
        for action_name in actions_to_be_shown:
            if action_name in name:
                print(name+r" right")        
                refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_right[name] )




In [ ]:
#this takes a while
grf_ref_data = refdata.GRFWalkingRefData()

In [ ]:
conv_names = {}
conv_names.update(refdata.graph_params.generate_grf_conv_names(0,weight))
conv_names.update(refdata.graph_params.generate_grf_conv_names(1,weight))

all_grf_curves_for_this_person = refdata.generate_action_plots([], xy_clippings_both, 
                                                           ref=grf_ref_data,
                                                skip_trials=skip_trials,
                                                action=this_action_name, include_actions=actions_to_be_shown,
                                                conv_names=conv_names)

In [ ]:

print("Std plot")


_ = refdata.plot_std_plots(all_grf_curves_for_this_person, plot_std=True, ref=grf_ref_data,
                       subplot_grid = (3,3),)
refdata.plt.show()
plt.savefig(f'sub{subject_num}_cop_id_{this_action_name}3x3_tighter_stds.png', bbox_inches = 'tight')



In [ ]:
print("Every curve plot")

_ = refdata.plot_std_plots(all_grf_curves_for_this_person, plot_std=False, ref=grf_ref_data,
                       subplot_grid = (3,3),)
plt.savefig(f'sub{subject_num}_cop_id_{this_action_name}3x3_tighter_all.png', bbox_inches = 'tight')

# IK from ID plots:

In [ ]:
action_trials= []
for trial in ik_files:
    for action_name in actions_to_be_shown:
        if action_name in trial:
            action_trials.append(trial)
action_trials #= ["/home/frekle/github/opensimrt/catkin_ws/tmp/02/48_s2_id_ramp2stair_filtered_1tau.sto"]

In [ ]:
xy_knees_ik = refdata.generate_somejoint_or_muscle_curves(action_trials,[], curve_prefix="ankle_angle")

xy_knees_list0 = []
xy_knees_list1 = []
for name, xy_tuples in xy_time_clippings_left.items():
    xy_knees_list0.append(xy_tuples)    
for name, xy_tuples in xy_time_clippings_right.items():
    xy_knees_list1.append(xy_tuples)  

left_str_ = "xy_time_clippings_left = {\n"
for sg_list, (name, xy_tuples) in zip(xy_knees_list0,xy_knees_ik[0].items()):
    left_str_ +="\"%s\":%s,\n"%(name,sg_list)
left_str_ += "}\n"
right_str_ = "xy_time_clippings_right = {\n"
for sg_list, (name, xy_tuples) in zip(xy_knees_list1,xy_knees_ik[1].items()):
    right_str_ += "\"%s\":%s,\n"%(name,sg_list)
right_str_ += "}\n"



In [ ]:
## Only run once. fix in the cell below
refdata.create_new_cell_below(left_str_)

In [ ]:
# manual corrections (shouldnt be necessary!):

for name, xy_tuples in xy_knees_ik[0].items():
    if name == action_trials[0]:
        #plt.title(name)        
        print(name)
        
        refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_left[name] )

In [ ]:
## Only run once. fix in the cell below
refdata.create_new_cell_below(right_str_)

In [ ]:
# manual corrections (shouldnt be necessary!):

for name, xy_tuples in xy_knees_ik[1].items():
    if name == action_trials[0]:
        #plt.title(name)
        print(name)
        refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_right[name] )

In [ ]:
## show all data that will be included

xy_clippings_both_ik = (xy_time_clippings_left,xy_time_clippings_right)

for name, xy_tuples in xy_knees_ik[0].items():    
    for action_name in actions_to_be_shown:
        if action_name in name:
            print(name+r" left")        
            refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_left[name] )
for name, xy_tuples in xy_knees_ik[1].items():
    for action_name in actions_to_be_shown:
        if action_name in name:
            print(name+r" right")        
            refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_right[name] )

In [ ]:
all_ik_curves_for_this_person = refdata.generate_action_plots(action_trials, xy_clippings_both_ik, ref=None, #refdata.GaitIKRefData(),
                                            skip_trials=skip_trials,
                                            action=this_action_name, include_actions=actions_to_be_shown,
                                            conv_names=refdata.graph_params.get_ik_graph_params())

In [ ]:
_ = refdata.plot_std_plots(all_ik_curves_for_this_person, plot_std=True, ref=refdata.GaitIKRefData(),subplot_grid = (3,3),)


In [ ]:
_ = refdata.plot_std_plots(all_ik_curves_for_this_person, plot_std=False, ref=None, #refdata.GaitIKRefData(),
                       subplot_grid = (3,3),)


In [ ]:
short_ik_curves_for_this_person = refdata.generate_action_plots([], xy_clippings_both_ik, ref=None, #refdata.GaitIKRefData(),
                                            skip_trials=skip_trials,
                                            action=this_action_name, include_actions=actions_to_be_shown,
                                            conv_names=refdata.graph_params.get_ik_short_graph_params())

_ = refdata.plot_std_plots(short_ik_curves_for_this_person, plot_std=True, ref=refdata.GaitIKRefData(),
                       subplot_grid = (1,3),)


## ID curve plotting:

In [ ]:
action_trials= []
for trial in id_files:
    for action_name in actions_to_be_shown:
        if action_name in trial:
            action_trials.append(trial)
action_trials #= ["/home/frekle/github/opensimrt/catkin_ws/tmp/02/48_s2_id_walking_filtered_1tau.sto"]

In [ ]:
xy_knees_id = refdata.generate_somejoint_or_muscle_curves(action_trials,[], curve_prefix="ankle_angle")


xy_knees_list0 = []
xy_knees_list1 = []
for name, xy_tuples in xy_time_clippings_left.items():
    xy_knees_list0.append(xy_tuples)    
for name, xy_tuples in xy_time_clippings_right.items():
    xy_knees_list1.append(xy_tuples)  

left_str_ = "xy_time_clippings_left = {\n"
for sg_list, (name, xy_tuples) in zip(xy_knees_list0,xy_knees_id[0].items()):
    left_str_ +="\"%s\":%s,\n"%(name,sg_list)
left_str_ += "}\n"
right_str_ = "xy_time_clippings_right = {\n"
for sg_list, (name, xy_tuples) in zip(xy_knees_list1,xy_knees_id[1].items()):
    right_str_ += "\"%s\":%s,\n"%(name,sg_list)
right_str_ += "}\n"



In [ ]:
## Only run once. fix in the cell below
refdata.create_new_cell_below(left_str_)

In [ ]:

for name, xy_tuples in xy_knees_id[0].items():
    if name == action_trials[0]:
        #plt.title(name)        
        print(name)
        
        refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_left[name] )


In [ ]:
## Only run once. fix in the cell below
refdata.create_new_cell_below(right_str_)

In [ ]:

for name, xy_tuples in xy_knees_id[1].items():
    if name == action_trials[0]:
        #plt.title(name)
        print(name)
        refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_right[name] )

In [ ]:
xy_clippings_both_id = (xy_time_clippings_left,xy_time_clippings_right)
print(xy_clippings_both[0])
print(xy_clippings_both[1])

for name, xy_tuples in xy_knees_id[0].items():    
    for action_name in actions_to_be_shown:
        if action_name in name:
            print(name+r" left")        
            refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_left[name] )
for name, xy_tuples in xy_knees_id[1].items():
    for action_name in actions_to_be_shown:
        if action_name in name:
            print(name+r" right")        
            refdata.clip_curve_test(xy_tuples[0],xy_tuples[1], time_clips = xy_time_clippings_right[name] )

In [ ]:
all_id_curves_for_this_person = refdata.generate_action_plots([], xy_clippings_both_id, 
                                                           ref=refdata.IdData(this_action_name),
                                                skip_trials=skip_trials,
                                                action=this_action_name, include_actions=actions_to_be_shown,
                                                conv_names=refdata.graph_params.get_id_all_graph_params(weight))

In [ ]:
_ = refdata.plot_std_plots(all_id_curves_for_this_person, plot_std=True, ref=refdata.IdData(this_action_name),
                       subplot_grid = (4,3),)



## This actually needs pickling and combining. no idea where that code is, need to find it, maybe under some ramp copy?

In [ ]:
reload(refdata.graph_params)
sagittal_id_curves_for_this_person = refdata.generate_action_plots([], xy_clippings_both_id, 
                                                           ref=refdata.IdData(this_action_name),
                                                skip_trials=skip_trials,
                                                action=this_action_name, include_actions=actions_to_be_shown,
                                                conv_names=refdata.graph_params.get_id_graph_params(weight))

_ = refdata.plot_std_plots(sagittal_id_curves_for_this_person, plot_std=True, ref=refdata.IdData(this_action_name),
                       subplot_grid = (1,3),)

plt.savefig(f'sub{subject_num}_id_sagittal_{this_action_name}_norefs_1x3_std.png', bbox_inches = 'tight')

## This would go on paper for other movements. see pickling comment above

In [ ]:
_ = refdata.plot_std_plots(sagittal_id_curves_for_this_person, plot_std=False, ref=refdata.IdGaitData(this_action_name),
                       subplot_grid = (1,3),)

plt.savefig(f'sub{subject_num}_id_sagittal_{this_action_name}_gaitrefs_1x3_all.png', bbox_inches = 'tight')

In [ ]:
#refdata.plot_std_plots(all_id_curves_for_this_person, plot_std=False, ref=refdata.IdGaitData(this_action_name),
_ = refdata.plot_std_plots(all_id_curves_for_this_person, plot_std=False, ref=refdata.IdData(this_action_name),
                       subplot_grid = (4,3))


# SO curves:

In [ ]:
so_files = glob.glob("*so.sto")
so_files.sort()
so_files

In [ ]:
so_action_trials= []
for trial in so_files:
    for action_name in actions_to_be_shown:
        if action_name in trial:
            so_action_trials.append(trial)

In [ ]:
xy_knees_so = refdata.generate_somejoint_or_muscle_curves(so_action_trials, [], curve_prefix="glut_med1", conv_names=refdata.graph_params.get_so_graph_params())

In [ ]:
xy_knees_list0 = []
xy_knees_list1 = []
for name, xy_tuples in xy_time_clippings_left.items():
    xy_knees_list0.append((name,xy_tuples))    
for name, xy_tuples in xy_time_clippings_right.items():
    xy_knees_list1.append((name,xy_tuples))    

In [ ]:
left_so_str = "xy_time_clippings_so_left = {\n"
for i, (name, xy_tuples) in enumerate(xy_knees_so[0].items()):
    left_so_str += "\""+name+"\":%s,\n"%str(xy_knees_list0[i][1])
left_so_str +="}\n"
right_so_str = "xy_time_clippings_so_right = {\n"
for i,(name, xy_tuples) in enumerate(xy_knees_so[1].items()):
    right_so_str +="\""+name+"\":%s,\n"%str(xy_knees_list1[i][1])
right_so_str +="}\n"

refdata.create_new_cell_below(left_so_str+right_so_str)

In [ ]:
xy_so_clippings_both = (xy_time_clippings_so_left,xy_time_clippings_so_right)
print(xy_so_clippings_both[0])
print(xy_so_clippings_both[1])

## Generates curves for all muscles

In [ ]:
all_so_curves_for_this_person = refdata.generate_action_plots(so_action_trials, xy_so_clippings_both, ref=None,
                                            skip_trials=skip_trials,
                                            action=this_action_name, include_actions=actions_to_be_shown,
                                            conv_names=refdata.graph_params.get_so_graph_params())

## Generates curves for main calf muscles

In [ ]:
selected_curves_for_this_person = refdata.generate_action_plots(so_action_trials, xy_so_clippings_both, ref=None,
                                            skip_trials=skip_trials,
                                            action=this_action_name, include_actions=actions_to_be_shown,
                                            conv_names=refdata.graph_params.get_so_short_graph_params())

## Generates plots for every trial and mean std curves for SO

In [ ]:
_ = refdata.plot_std_plots(selected_curves_for_this_person, plot_std=False, ref=refdata.SoData(this_action_name),
                       subplot_grid = (2,3),)


In [ ]:
plt.rcParams['figure.figsize'] = [8, 10]
_ = refdata.plot_std_plots(selected_curves_for_this_person, plot_std=True, ref=refdata.SoData(this_action_name),
                       subplot_grid = (2,3),)

In [ ]:
reload(refdata)
plt.rcParams['figure.figsize'] = [8, 10]
axcurve_list = refdata.creat_axs(all_ik_curves_for_this_person, ref= refdata.GaitIKRefData(this_action_name))
fig, ax =  refdata.create_axs_dimensions(3,3) 
fig, ax, nl, legend = refdata.plotAX(axcurve_list, ax, fig)

fig.legend(nl, legend,loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
axcurve_list_id2 = refdata.apply_offset_to_axs(refdata.creat_axs(all_id_curves_for_this_person),3)

axcurve_list.extend(axcurve_list_id2)

In [ ]:
fig, ax =  refdata.create_axs_dimensions(7,3) 

fig, ax, nl, legend = refdata.plotAX(axcurve_list, ax, fig)

In [ ]:
selected_curves_for_this_person_smaller = refdata.generate_action_plots(so_action_trials, xy_so_clippings_both, ref=None,
                                            skip_trials=skip_trials,
                                            action=this_action_name, include_actions=actions_to_be_shown,
                                            conv_names=refdata.graph_params.get_so_even_smaller())

In [ ]:
axcurve_list_so2 = refdata.apply_offset_to_axs(refdata.creat_axs(selected_curves_for_this_person_smaller
                                                                 ,ref=refdata.SoData(this_action_name)),0)

In [ ]:


fig, axs = refdata.create_axs_dimensions(1, 4, margin= 1.5, header = 1.2, subheigth = 6, subwidth= 6)

#refdata.plt.rcParams['figure.figsize'] = [12, 4]
#axcurve_list.extend(axcurve_list_so2)
fig, ax, nl, legend = refdata.plotAX(axcurve_list_so2, axs, fig)


In [ ]:
axcurve_listXXXik = refdata.creat_axs(short_ik_curves_for_this_person, ref= refdata.GaitIKRefData(this_action_name))

axcurve_listXXXid = refdata.apply_offset_to_axs(refdata.creat_axs(sagittal_id_curves_for_this_person),1)

axcurve_listXXXso = refdata.apply_offset_to_axs(refdata.creat_axs(selected_curves_for_this_person
                                                                 ,ref=refdata.SoData(this_action_name)),2)

axcurve_listXXXik.extend(axcurve_listXXXid)
axcurve_listXXXik.extend(axcurve_listXXXso)



## This is the sagital plane with calf muscles and stds goes in paper

In [ ]:
import matplotlib as mpl
from matplotlib.legend_handler import HandlerLine2D, HandlerTuple
mpl.rcParams['figure.dpi'] = 300

reload(refdata)
fig, axs = refdata.create_axs_dimensions(4, 3, margin= 1.5, header = 1.2, subheigth = 6, subwidth= 6)
for ax in axs.flatten():
    ax.set_axis_off()

fig, ax, nh, nl = refdata.plotAX(axcurve_listXXXik, axs, fig, plot_ref_curves=True)
#fig.legend(nh[0:2], ["Left","Right"],loc='lower right', bbox_to_anchor=(0.85, 0.18))#, loc='lower right')
#fig.legend(nh, nl,loc='lower right', bbox_to_anchor=(0.85, 0.5))#, loc='lower right')
l = fig.legend([(nh[1],nh[2],nh[3]),(nh[4],nh[5],nh[6]),(nh[0], )], ["Left $\pm$ 1 sd","Right $\pm$ 1 sd","Ref. mean $\pm$ 1 sd"], numpoints=1,
              handler_map={tuple: HandlerTuple(ndivide=None)},loc='lower right', bbox_to_anchor=(0.9, 0.2))

plt.savefig(f'sub{subject_num}_ik_id_so_{this_action_name}4x3_tighter_stds.png', bbox_inches = 'tight')

## This is the sagital plane with calf muscles with all. maybe goes in appendix paper

In [ ]:
fig, axs = refdata.create_axs_dimensions(4, 3, margin= 1.5, header = 1.2, subheigth = 6, subwidth= 6)
for ax in axs.flatten():
    ax.set_axis_off()

reload(refdata)
fig, ax, nh, nl = refdata.plotAX(axcurve_listXXXik, axs, fig, plot_ref_curves=True, plot_std=False, legend=True)
print(nl)
#fig.legend(nh[0:2], ["Left","Right"],loc='lower right', bbox_to_anchor=(0.85, 0.18))#, loc='lower right')
#fig.legend(nh, nl,loc='lower right', bbox_to_anchor=(0.85, 0.5))#, loc='lower right')
#l = fig.legend([(nh[1],nh[2],nh[3]),(nh[4],nh[5],nh[6]),(nh[0], )], ["Left $\pm$ 1 sd","Right $\pm$ 1 sd","Ref. mean $\pm$ 1 sd"], numpoints=1,
#              handler_map={tuple: HandlerTuple(ndivide=None)},loc='lower right', bbox_to_anchor=(0.9, 0.2))

plt.savefig(f'sub{subject_num}_ik_id_so_{this_action_name}4x3_tighter_all.png', bbox_inches = 'tight')
!nautilus .

In [ ]:
axcurve_list_so3 = refdata.apply_offset_to_axs(refdata.creat_axs(selected_curves_for_this_person_smaller
                                                                 ,ref=refdata.SoData(this_action_name)),2)
axcurve_list.extend(axcurve_list_so3)

In [ ]:
axcurve_list__ = refdata.creat_axs(short_ik_curves_for_this_person, ref= refdata.GaitIKRefData(this_action_name))

axcurve_list___ = refdata.apply_offset_to_axs(refdata.creat_axs(all_id_curves_for_this_person, ref=refdata.IdGaitData(this_action_name)),1)

axcurve_list____ = refdata.apply_offset_to_axs(refdata.creat_axs(selected_curves_for_this_person_smaller
                                                                 ,ref=refdata.SoData(this_action_name)),4)

axcurve_list__.extend(axcurve_list___)
axcurve_list__.extend(axcurve_list____)


In [ ]:
fig, ax =  refdata.create_axs_dimensions(5,4)

fig, ax, nl, legend = refdata.plotAX(axcurve_list__, ax, fig)

In [ ]:

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

reload(refdata)
fig, axs = refdata.create_axs_dimensions(5, 4, margin= 1.5, header = 1.5, subheigth = 6, subwidth= 6)
for ax in axs.flatten():
    ax.set_axis_off()

fig, ax, nh, nl = refdata.plotAX(axcurve_list__, axs, fig, plot_ref_curves=True)
print(nl)
#fig.legend(nh[0:2], ["Left","Right"],loc='lower right', bbox_to_anchor=(0.85, 0.18))#, loc='lower right')
#fig.legend(nh, nl,loc='lower right', bbox_to_anchor=(0.85, 0.5))#, loc='lower right')
l = fig.legend([(nh[3],nh[4],nh[5]),
                (nh[7],nh[8],nh[9]),
                (nh[2], ),
                (nh[0],nh[1],nh[6])], 
               ["Left $\pm$ 1 sd",
                "Right $\pm$ 1 sd",
                "Ref. mean $\pm$ 1 sd",
                "mean $\pm$ 1sd"], numpoints=1,
              handler_map={tuple: HandlerTuple(ndivide=None)},loc='lower right', bbox_to_anchor=(0.9, 0.5))

In [ ]:
fig, axs = refdata.create_axs_dimensions(5, 4, margin= 1.5, header = 1.5, subheigth = 6, subwidth= 6)
for ax in axs.flatten():
    ax.set_axis_off()

fig, ax, nh, nl = refdata.plotAX(axcurve_list__, axs, fig)
#fig.legend(nh[0:2], ["Left","Right"],loc='lower right', bbox_to_anchor=(0.85, 0.18))#, loc='lower right')
#fig.legend(nh, nl,loc='lower right', bbox_to_anchor=(0.85, 0.5))#, loc='lower right')
l = fig.legend([(nh[1],nh[2],nh[3]),(nh[4],nh[5],nh[6]),(nh[0], )], ["Left $\pm$ 1 sd","Right $\pm$ 1 sd","Ref. mean $\pm$ 1 sd"], numpoints=1,
              handler_map={tuple: HandlerTuple(ndivide=None)},loc='lower right', bbox_to_anchor=(0.9, 0.5))

## generate graphs from id or so events

In [ ]:
import re

#which_bag_to_parse = "id"
which_bag_to_parse = "so"

def escape_special_characters(input_string):
    # Define a dictionary of characters to be escaped
    escape_dict = {
        "&": r"\&",
        "%": r"\%",
        "$": r"\$",
        "#": r"\#",
        "_": r"\_",
        "{": r"\{",
        "}": r"\}",
        "^": r"\^{}",
        "~": r"\textasciitilde{}",
        "\\": r"\textbackslash{}"
    }
    
    # Use regular expressions to find and escape the characters
    escaped_string = re.sub(r'([&%$#_{}^~\\])', lambda x: escape_dict[x.group()], input_string)
    
    return escaped_string

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [16, 12]
def eventnames(df):
    d = {}
    for coln in df.columns:
        if "field.header.stamp" == coln:
            #real_name = df["field.header.frame_id"]
            real_name = "ik produced"
            d.update({coln: real_name})
        if "stamp" in coln and "event" in coln:
            real_name_col = ".".join(coln.split(".")[0:3])+".name"
            real_name = df[real_name_col][0]
            d.update ({coln: real_name})
    #for coln, real_name in d.items():
    #    print("%s -> %s"%(coln, real_name))
    return d
#wanted_cols = eventnames(df)

def get_cols(df):
    wanted_cols = eventnames(df)
    a = []
    for v in wanted_cols:
        a.append(v)
        #print(v)
    return a, wanted_cols

def rename_df(df):
    a = get_cols(df)
    return df[a[0]].rename(columns=a[1])

def plot_differences(df):
    
    fig, ax = plt.subplots()
    for i, col in enumerate(df.columns):
        if i == 0:
            continue
        this_column = df.iloc[:,i]
        #print(this_column)
        ref_column = df.iloc[:,i-1]
        #print(ref_column)
        
        y = this_column -ref_column
        y_label = df.columns[i] +" - " + df.columns[i-1]
        print("%s:\t %f ms"%(y_label+" mean", np.mean(y)/1000000))
        y_label_cleaned = escape_special_characters(y_label)
        ax.plot(y, label= y_label_cleaned)
    ax.legend()

bag_files = glob.glob("%s*.bag_timings.txt"%which_bag_to_parse)    

for file_i in bag_files:
    df = pd.read_csv(file_i)
    a = rename_df(df)
    plot_differences(a)
    plt.show()

# Saving figures part

This is old, needs to be updated before being used!

In [ ]:
import pickle


In [ ]:
#subject_data_directory = os.getcwd().split("experiment_data/")[0]+"experiment_data/"
subject_data_directory = os.getcwd()

In [ ]:
_filename = f"{subject_num}_{this_action_name}_allcurves.pkl"

subject_complete_filename = os.path.join(subject_data_directory, _filename)
print(subject_complete_filename)
f = open(subject_complete_filename,"wb")
pickle.dump(all_ik_curves_for_this_person,f)
f.close()